# Import necessary libraries


In [29]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import re

pd.options.display.max_columns = 200
pd.options.display.max_rows = 3000

# Load the dataset 

In [48]:
def parent_directory():

  # Create a relative path to the parent
  # of the current working directory
  relative_parent = os.path.join(os.getcwd(), os.pardir)
  # Return the absolute path of the parent directory
  return os.path.abspath(relative_parent)

print(parent_directory())

def str_2_int(value):
  if ',' in value:
    x = float(value.replace(',',''))
    return x
  else :
    return float(value)

salaries = pd.read_csv(parent_directory() + '/final_project/0.Ressources/05_2_salaries.csv')


c:\Users\33668\Documents\Albert school Msc1\Pandas


# Renaming Columns and Adding Total Remuneration

In [31]:


#Rename Column names because they were too long
salaries.columns = ['Time', 'Age', 'Industry', 'Job', 'Job Context', 'Salary', 'Bonus', 'Currency', 'Other Currency', 'Income Context', 'Country', 'US State', 'City', 'Experience', 'Field experience', 'Education', 'Gender', 'Race']

#Convert String to float
salaries["Salary"] = salaries["Salary"].apply(str_2_int)


#Replace NA values in Bonus by 0 and add a "Total Remuneration" Column
values = {"Bonus": 0.0}
salaries = salaries.fillna(value = values)
salaries["Total Remuneration"] = salaries["Salary"] + salaries["Bonus"]
move = salaries.pop("Total Remuneration")
salaries.insert(7,"Total Remuneration" ,move)


salaries


,Time,Age,Industry,Job,Job Context,Salary,Bonus,Total Remuneration,Currency,Other Currency,Income Context,Country,US State,City,Experience,Field experience,Education,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000.0,0.0,55000.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600.0,4000.0,58600.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000.0,0.0,34000.0,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000.0,3000.0,65000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000.0,7000.0,67000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28080,9/25/2024 19:54:18,45-54,"Accounting, Banking & Finance",Product Manager Lead,NaN,117000.0,8000.0,125000.0,USD,NaN,NaN,USA,Missouri,Remote,11 - 20 years,11 - 20 years,College degree,Woman,White
28081,9/25/2024 21:34:43,45-54,Education (Primary/Secondary),Curriculum Writer,I am a freelance contract curriculum writer fo...,70000.0,0.0,70000.0,USD,NaN,NaN,United States,South Carolina,Bennettsville,21 - 30 years,21 - 30 years,Master's degree,Woman,White
28082,9/30/2024 10:52:30,55-64,Government and Public Administration,Clerical Officer,NaN,28600.0,0.0,28600.0,EUR,NaN,NaN,Ireland,NaN,Dublin,21 - 30 years,1 year or less,"Professional degree (MD, JD, etc.)",Man,White
28083,10/4/2024 13:05:24,18-24,Computing or Tech,Software Engineering Co-Op,I was an Intern,56160.0,0.0,56160.0,USD,NaN,NaN,United States,Rhode Island,Jhonston,1 year or less,1 year or less,College degree,Man,White


# Fixing Types

In [32]:
salaries["Time"] = pd.to_datetime(salaries["Time"], format='%m/%d/%Y %H:%M:%S')

salaries.dtypes

Time                  datetime64[ns]
Age                           object
Industry                      object
Job                           object
Job Context                   object
Salary                       float64
Bonus                        float64
Total Remuneration           float64
Currency                      object
Other Currency                object
Income Context                object
Country                       object
US State                      object
City                          object
Experience                    object
Field experience              object
Education                     object
Gender                        object
Race                          object
dtype: object

# Check Missing Values

In [33]:
pair_list = salaries["Currency"].unique()

In [34]:
missing_values = salaries.isnull().sum()
missing_percentage = (missing_values / len(salaries)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})
missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage (%)', ascending=False)

,Missing Values,Percentage (%)
Other Currency,27878,99.262952
Income Context,25041,89.161474
Job Context,20818,74.124978
US State,5026,17.895674
Education,222,0.790458
Race,177,0.630230
Gender,171,0.608866
City,82,0.291971
Industry,74,0.263486
Job,1,0.003561


We can therefore see that over 99.2% of the dataset is payed in one of the currencies listed in the "Currency" column. For this reason we think it would make sense to remove the other currencies from our analysis as they would not have a significant impact on the results. We will therefore analyse salaries from the US, Great Britain, Canada, Euro Zone, Australia and New Zealand, Switzerland, South Africa, Sweden, Hong Kong and Japan. This may bias our views due to it being focused on advanced economies.   

# Get current exchange rate

In [35]:
#We are going to use Selenium here to get the exchange rate of the currencies
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# instantiate a Chrome options object
options = webdriver.ChromeOptions()

# set the options to use Chrome in headless mode
options.add_argument("--headless=new")

# initialize an instance of the chrome driver (browser) in headless mode
driver = webdriver.Chrome(
    options=options,
)

# visit your target site
driver.get("https://www.oanda.com/currency-converter/live-exchange-rates/")

# extract all the product containers
rates = driver.find_elements(By.XPATH, "/html/body/div[1]/main/div/div/div[3]")

time.sleep(3)

In [36]:
def get_rate(rate):
  if rate == "EUR":
    return 1
  
  #HKD doesnt exist unfortunately and I couldn't find it so i will unfortunately hard code the rate
  if rate == "HKD":
    return 0.12
  
  if rate == "AUD/NZD":
    rate = "AUD"
  
  rate_pair = rate + "/EUR"
  for rate in rates:
    if rate_pair in rate.text:
      exchange_rate_path = rate.text.split(rate_pair)[1].split("\n")
      return float(exchange_rate_path[1] + exchange_rate_path[2])

pair_list = salaries['Currency'].unique().tolist()

exchange_rate = {}
for pair in pair_list:
  exchange_rate[pair] = get_rate(pair)



exchange_rate

{'USD': 0.9331,
 'GBP': 1.2002,
 'CAD': 0.6697,
 'EUR': 1,
 'AUD/NZD': 0.6132,
 'Other': None,
 'CHF': 1.0648,
 'ZAR': 0.0527,
 'SEK': 0.0857,
 'HKD': 0.12,
 'JPY': 0.006}

# Voici ton code intial sur currency rate conversion

In [37]:
salaries["Other Currency"] = salaries['Other Currency'].fillna('NaN')
allowed_currencies = ['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'CHF', 'ZAR', 'SEK', 'HKD', 'JPY']
other = ['Other', 'NaN', 'Na', 'N/a']
salaries = salaries[
    (salaries['Currency'].isin(allowed_currencies) & salaries['Other Currency'].isin(other) ) | (salaries['Currency'].isin(other) & salaries['Other Currency'].isin(allowed_currencies))]
print(salaries['Currency'].unique())
print(salaries['Other Currency'].unique())
len(salaries)


# prompt: make a function that converts the values of the "Total Remuneration" into EUR depending on the currency located in "Currency" or "Other currency" with current exchange rates

def convert_to_eur(row):
  """Converts the 'Total Remuneration' to EUR based on the currency in 'Currency' or 'Other Currency'.

  Args:
    row: A pandas Series representing a row in the DataFrame.

  Returns:
    The 'Total Remuneration' in EUR.
  """
  #Current Exchange Rates
  exchange_rates = {
      'USD': 0.93,  # EUR/USD rate
      'GBP': 1.15,  # EUR/GBP rate
      'CAD': 0.68,  # EUR/CAD rate
      'EUR': 1.00,  # EUR/EUR rate
      'AUD/NZD': 0.60,  # EUR/AUD rate (assuming the same for NZD for simplicity)
      'CHF': 0.95,  # EUR/CHF rate
      'ZAR': 0.05,  # EUR/ZAR rate
      'SEK': 0.09,  # EUR/SEK rate
      'HKD': 0.12,  # EUR/HKD rate
      'JPY': 0.0071,  # EUR/JPY rate
  }

  if row['Currency'] in exchange_rates:
    currency = row['Currency']
  elif row['Other Currency'] in exchange_rates:
    currency = row['Other Currency']
  else:
    return None  # Return None if currency is not found

  return row['Total Remuneration'] * exchange_rates[currency]

# Apply the function to create a new column
salaries['Total Remuneration EUR'] = salaries.apply(convert_to_eur, axis=1)
salaries

['USD' 'GBP' 'CAD' 'EUR' 'AUD/NZD' 'CHF' 'ZAR' 'SEK' 'Other' 'HKD' 'JPY']
['NaN' 'CHF' 'USD' 'SEK' 'EUR' 'GBP' 'ZAR' 'CAD' 'N/a']


C:\Users\33668\AppData\Local\Temp\ipykernel_29656\2208309286.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salaries['Total Remuneration EUR'] = salaries.apply(convert_to_eur, axis=1)


,Time,Age,Industry,Job,Job Context,Salary,Bonus,Total Remuneration,Currency,Other Currency,Income Context,Country,US State,City,Experience,Field experience,Education,Gender,Race,Total Remuneration EUR
0,2021-04-27 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000.0,0.0,55000.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White,51150.0
1,2021-04-27 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600.0,4000.0,58600.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,67390.0
2,2021-04-27 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000.0,0.0,34000.0,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,31620.0
3,2021-04-27 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000.0,3000.0,65000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,60450.0
4,2021-04-27 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000.0,7000.0,67000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,62310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28080,2024-09-25 19:54:18,45-54,"Accounting, Banking & Finance",Product Manager Lead,NaN,117000.0,8000.0,125000.0,USD,NaN,NaN,USA,Missouri,Remote,11 - 20 years,11 - 20 years,College degree,Woman,White,116250.0
28081,2024-09-25 21:34:43,45-54,Education (Primary/Secondary),Curriculum Writer,I am a freelance contract curriculum writer fo...,70000.0,0.0,70000.0,USD,NaN,NaN,United States,South Carolina,Bennettsville,21 - 30 years,21 - 30 years,Master's degree,Woman,White,65100.0
28082,2024-09-30 10:52:30,55-64,Government and Public Administration,Clerical Officer,NaN,28600.0,0.0,28600.0,EUR,NaN,NaN,Ireland,NaN,Dublin,21 - 30 years,1 year or less,"Professional degree (MD, JD, etc.)",Man,White,28600.0
28083,2024-10-04 13:05:24,18-24,Computing or Tech,Software Engineering Co-Op,I was an Intern,56160.0,0.0,56160.0,USD,NaN,NaN,United States,Rhode Island,Jhonston,1 year or less,1 year or less,College degree,Man,White,52228.8


# Questions
1. Which industry pays the most?
2. How does salary increase given years of experience?
3. How do salaries compare for the same role in different locations?
4. How much do salaries differ by gender and years of experience?
5. How do factors like race and education level correlate with salary?
6. Is there a “sweet spot” total work experience vs years in the specific field?

In [38]:
salary_average = salaries[['Industry','Total Remuneration EUR']]
list_pay_industry = salary_average.groupby("Industry").mean()
print(list_pay_industry.sort_values(by="Total Remuneration EUR", ascending= False))



                                                    Total Remuneration EUR
Industry                                                                  
Accounting, Banking & Finance                                 2.361460e+06
Social networks                                               1.860000e+06
Commercial Building Material Distribution                     5.115000e+05
Sports                                                        3.720000e+05
Finance                                                       3.577012e+05
Food & Beverage                                               3.411550e+05
Biotech / life sciences                                       3.217800e+05
Biotech/pharmaceuticals                                       3.180600e+05
pharma / medical device design and manufacturing              2.678400e+05
Energy (oil & gas & associated products, renewa...            2.653290e+05
Pharmaceutical/biotechnology                                  2.645850e+05
Petroleum                